In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import json
import os
import pandas as pd
from datetime import datetime as dt
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [2]:
# Loop through cities
tweet_dir= Path("../data/processed/park_user_tweets_0530")
cities = list(tweet_dir.glob("*.json"))

## load all stop wordds
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)

In [3]:
stop_dict_all = stop_dict['all_cities']
word_table = {}
for city, words in stop_dict.items():
    code =city[3:-8].replace("_",' ')
    words = ', '.join([x for x in words if x not in stop_dict_all])
    if len(words) > 0:
        word_table[code] = words
word_table = pd.DataFrame.from_dict(word_table, 'index')

In [20]:
word_table.to_latex('results/words/city_stops.tex')

In [4]:
city_tz = pd.read_csv("../data/processed/timezones/city_timezone.csv", index_col='city').to_dict('dict')['tz']
#timezones https://en.wikipedia.org/wiki/Time_in_Indiana 
#https://en.wikipedia.org/wiki/List_of_tz_database_time_zones
eastern = "US/Eastern"
pacific = "US/Pacific"
arizona = "America/Phoenix"
mountain = "US/Mountain"
central = "US/Central"

In [5]:
def tweets_timelabel(park_user_tweets, timezone="US/Eastern"):
    tweets = get_tweets_timestamp(park_user_tweets)
    tweet_df = pd.DataFrame.from_dict(tweets,dtype=str)
    tweet_df['tweet_created_at'] = pd.to_datetime(tweet_df['tweet_created_at'],format="%Y-%m-%d %H:%M:%S")
    tweet_df['dt_local'] =  tweet_df['tweet_created_at'].dt.tz_localize('utc').dt.tz_convert(timezone)
    tweet_df['hour_of_day'] = tweet_df['dt_local'].dt.hour
    tweet_df['day_of_week'] = tweet_df['dt_local'].dt.day_name()
    tweet_df['year'] = tweet_df['dt_local'].dt.year
    tweet_df['season'] = tweet_df['dt_local'].dt.month.apply(lambda month:(month%12 + 3)//3)
    tweet_df['season'].replace({1:"winter",2:"spring",3:"summer",4:"fall"}, inplace=True)
    return tweet_df

In [6]:
binner='season'
cats = ['fall', 'summer', 'winter', 'spring']
park_tweets_bytype = {cat:[] for cat in cats}
control_tweets_bytype = {cat:[] for cat in cats}
park_stops = []

In [7]:
for city in cities:
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    print("City: {} w {} park users".format(city, len(park_user_tweets)))
    tweet_df = tweets_timelabel(park_user_tweets, pacific)
    park_list = list(set(tweet_df.park_name))
    park_stops += get_park_stopwords(park_list)
    for cat in cats:
        tweet_subset = tweet_df[tweet_df[binner] ==cat]
        index_list = range(len(tweet_subset))
        n = min(1000, len(tweet_subset))
        sample = random.sample(index_list, n)
        # print("City: {} -{} {} tweets".format(city, cat, n))
        park_text = list(tweet_subset.park_text.values)
        control_text = list(tweet_subset.control_text.values)
        park_tweet_sample = [park_text[i] for i in sample]
        control_tweet_sample = [control_text[i] for i in sample]
        park_tweets_bytype[cat]+=park_tweet_sample
        control_tweets_bytype[cat]+=control_tweet_sample

City: ../data/processed/park_user_tweets_0530/CO_Denver_0820000.json w 3902 park users
City: ../data/processed/park_user_tweets_0530/AZ_Phoenix_0455000.json w 7566 park users
City: ../data/processed/park_user_tweets_0530/FL_Jacksonville_1235000.json w 3218 park users
City: ../data/processed/park_user_tweets_0530/TX_Austin_4805000.json w 14689 park users
City: ../data/processed/park_user_tweets_0530/TX_Fort_Worth_4827000.json w 4278 park users
City: ../data/processed/park_user_tweets_0530/TX_El_Paso_4824000.json w 1397 park users
City: ../data/processed/park_user_tweets_0530/CA_San_Diego_0666000.json w 22269 park users
City: ../data/processed/park_user_tweets_0530/TN_Memphis_4748000.json w 3112 park users
City: ../data/processed/park_user_tweets_0530/DC_Washington_1150000.json w 41062 park users
City: ../data/processed/park_user_tweets_0530/WA_Seattle_5363000.json w 7739 park users
City: ../data/processed/park_user_tweets_0530/IL_Chicago_1714000.json w 36919 park users
City: ../data/pro

In [8]:
# check length of < 1 ..park_tweets_bytype = {cat:[] for cat in cats}
for park_type,tweets in park_tweets_bytype.items():
    print(park_type,len(tweets))

len(park_stops)

fall 20982
summer 22730
winter 21382
spring 22898


12629

In [10]:
for cat in cats:
    with open("../data/processed/time_tweets/time_tweets_0605/park_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(park_tweets_bytype[cat],fp)
    
    with open("../data/processed/time_tweets/time_tweets_0605/time_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(control_tweets_bytype[cat],fp)

In [11]:
## load frequent stop words  stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)
    
#load stop words selected from raw wordshifts for probelmatic words
freq_stops = []
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

# combine with  names of parks
all_stops = freq_stops + park_stops


# put through ngram filter, lower
all_stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(all_stops), path='../src/func/ngrams.bin')).keys()]))
print(len(all_stops))
# select ones with valence below 4 or greater than 6
stops_w_val=[]
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
            #print(stop,senti_dict[stop])
# check how #many of stop words are in labmt outside 4-6 range
print(len(stops_w_val))

6969
538


In [12]:
results_bytype ={}
for cat in cats:
    results_bytype[cat] =  bootstrap_sentiment(park_tweets_bytype[cat], control_tweets_bytype[cat], stops_w_val,sample=.8,runs=10)

In [13]:
results_bytype

{'fall': [0.0892321731802026,
  0.09273914772683067,
  0.092336844478913,
  0.0922857165848443,
  0.0879960163259188,
  0.10544741882965436,
  0.09698224415582501,
  0.08672222095585536,
  0.0724846822062366,
  0.08189164035924357],
 'summer': [0.12061366876074153,
  0.10920674437790634,
  0.12444016516923462,
  0.11935298311668507,
  0.12262197304191247,
  0.11088711823612396,
  0.10928853941832184,
  0.11260601667189274,
  0.11398938429753525,
  0.12380639835050644],
 'winter': [0.03637892432251455,
  0.04568758818766572,
  0.052808311049188816,
  0.04815781769147609,
  0.042569838593917275,
  0.04288115219313582,
  0.04354434836902943,
  0.03926473048051182,
  0.04973774248473273,
  0.04674372656678916],
 'spring': [0.06948189766521828,
  0.08400329145956142,
  0.08208108563253091,
  0.06885467183524163,
  0.07862857773813658,
  0.08436167099283498,
  0.07465485681875617,
  0.07791932808765178,
  0.07743087155175132,
  0.08557041289987044]}

In [14]:
with open("./results/city_combined_0605/bootstrap_{}.json".format(binner), 'w') as fp:
    json.dump(results_bytype,fp)

In [15]:
binner='day_of_week'
cats = ['Sunday', 'Friday', 'Monday', 'Saturday', 'Thursday', 'Wednesday', 'Tuesday']
park_tweets_bytype = {cat:[] for cat in cats}
control_tweets_bytype = {cat:[] for cat in cats}

for city in tweet_dir.glob("*.json"):
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    tweet_df = tweets_timelabel(park_user_tweets)
    park_list = list(set(tweet_df.park_name))
    park_stops += get_park_stopwords(park_list)

    for cat in cats:
        tweet_subset = tweet_df[tweet_df[binner] == cat]
        index_list = range(len(tweet_subset))
        n = min(1000, len(tweet_subset))
        sample = random.sample(index_list, n)
        park_text = list(tweet_subset.park_text.values)
        control_text = list(tweet_subset.control_text.values)
        park_tweet_sample = [park_text[i] for i in sample]
        control_tweet_sample = [control_text[i] for i in sample]
        park_tweets_bytype[cat]+=park_tweet_sample
        control_tweets_bytype[cat]+=control_tweet_sample

In [16]:
# check length of < 1 ..park_tweets_bytype = {cat:[] for cat in cats}
for park_type,tweets in park_tweets_bytype.items():
    print(park_type,len(tweets))

len(park_stops)

Sunday 19025
Friday 18006
Monday 17161
Saturday 20124
Thursday 17594
Wednesday 17480
Tuesday 17131


25244

In [17]:
for cat in cats:
    with open("../data/processed/time_tweets/time_tweets_0605/park_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(park_tweets_bytype[cat],fp)
    
    with open("../data/processed/time_tweets/time_tweets_0605/time_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(control_tweets_bytype[cat],fp)

In [18]:
## load frequent stop words  stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)
    
#load stop words selected from raw wordshifts for probelmatic words
freq_stops = [
    
]
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

# combine with  names of parks
all_stops = freq_stops + park_stops


# put through ngram filter, lower
all_stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(all_stops), path='../src/func/ngrams.bin')).keys()]))
print(len(all_stops))
# select ones with valence below 4 or greater than 6
stops_w_val=[]
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
            #print(stop,senti_dict[stop])
# check how #many of stop words are in labmt outside 4-6 range
print(len(stops_w_val))

7086
545


In [19]:
results_bytype ={}
for cat in cats:
    print(cat)
    results_bytype[cat] =  bootstrap_sentiment(park_tweets_bytype[cat], control_tweets_bytype[cat], stops_w_val,sample=.8,runs=10)

Sunday
Friday
Monday
Saturday
Thursday
Wednesday
Tuesday


In [30]:
binner

'day_of_week'

In [20]:
with open("./results/city_combined_0605/bootstrap_{}.json".format(binner), 'w') as fp:
    json.dump(results_bytype,fp)

In [21]:
binner='hour_of_day'
cats = range(24)
park_tweets_bytype = {cat:[] for cat in cats}
control_tweets_bytype = {cat:[] for cat in cats}

for city in tweet_dir.glob("*.json"):
    with open(city, 'r') as f:
        park_user_tweets = json.load(f)
    tweet_df = tweets_timelabel(park_user_tweets, city_tz[city.stem])

    park_list = list(set(tweet_df.park_name))
    park_stops += get_park_stopwords(park_list)

    for cat in cats:
        tweet_subset = tweet_df[tweet_df[binner] == cat]
        index_list = range(len(tweet_subset))
        n = min(1000, len(tweet_subset))
        sample = random.sample(index_list, n)
        park_text = list(tweet_subset.park_text.values)
        control_text = list(tweet_subset.control_text.values)
        park_tweet_sample = [park_text[i] for i in sample]
        control_tweet_sample = [control_text[i] for i in sample]
        park_tweets_bytype[cat]+=park_tweet_sample
        control_tweets_bytype[cat]+=control_tweet_sample

In [34]:
#for hour, tweets in park_tweets_bytype.items():
#    print(hour,len(tweets))

In [22]:
for cat in cats:
    with open("../data/processed/time_tweets/time_tweets_0605/park_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(park_tweets_bytype[cat],fp)
    
    with open("../data/processed/time_tweets/time_tweets_0605/control_tweets_{}_{}.json".format(binner,cat), 'w') as fp:
        json.dump(control_tweets_bytype[cat],fp)

In [23]:
## load frequent stop words  stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)
    
#load stop words selected from raw wordshifts for probelmatic words
freq_stops = []
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

# combine with  names of parks
all_stops = freq_stops + park_stops


# put through ngram filter, lower
all_stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(all_stops), path='../src/func/ngrams.bin')).keys()]))
print(len(all_stops))
# select ones with valence below 4 or greater than 6
stops_w_val=[]
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
            #print(stop,senti_dict[stop])
# check how #many of stop words are in labmt outside 4-6 range
print(len(stops_w_val))

7144
545


In [28]:
results_bytype ={}
n=5000

for cat in range(8,24):
    index_list = range(len(park_tweets_bytype[cat]))
    size = min(len(park_tweets_bytype[cat]),10000)
    sample = random.sample(index_list, size)
    park_tweet_sample = [park_tweets_bytype[cat][i] for i in sample]
    control_tweet_sample = [control_tweets_bytype[cat][i] for i in sample]
    results_bytype[cat] =  bootstrap_sentiment(park_tweet_sample,control_tweet_sample, stops_w_val,sample=.8,runs=10)

In [31]:
with open("./results/city_combined_0605/bootstrap_{}.json".format(binner), 'w') as fp:
    json.dump(results_bytype,fp)